In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
         pass
        

# Any results you write to the current directory are saved as output.

In [2]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [3]:
#loading the data
train_data = pd.read_csv("/kaggle/input/classification-of-images/dataset/train.csv")
test_data = pd.read_csv("/kaggle/input/classification-of-images/dataset/test.csv")

In [4]:
train_data.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [5]:
#Encoding the categorical values to numerical values
le = LabelEncoder()
train_data['Class']=le.fit_transform(train_data['Class'])
# train_data['Class']

In [6]:
#Separating the x_train and y_train
x_train=[]
y_train=[]
i=0
for img in train_data['Image']:
    path = os.path.join("/kaggle/input/classification-of-images/dataset/Train Images",img)
    train_img = cv2.imread(path)
    train_img=cv2.resize(train_img,(150,150))
    train_img = train_img.astype('float32')
    x_train.append(train_img)
    y_train.append(train_data['Class'][i])
    i=i+1
# print(x_train[0])


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Data Augmentation
datagen = ImageDataGenerator(
      featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False, 
        zca_whitening=False,
        rotation_range=10,
        zoom_range = 0.1, 
        width_shift_range=0.2,
        height_shift_range=0.2,  
        horizontal_flip=False,
        vertical_flip=True)  

datagen.fit(x_train)

In [8]:
x_test=[]
for img in test_data['Image']:
    path = os.path.join("/kaggle/input/classification-of-images/dataset/Test Images",img)
    test_img = cv2.imread(path)
    test_img=cv2.resize(test_img,(150,150))
    test_img = test_img.astype('float32')
    x_test.append(test_img)

In [9]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(5983, 150, 150, 3)
(5983,)
(3219, 150, 150, 3)


In [10]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, Activation, MaxPooling2D, BatchNormalization
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [11]:
num_classes = 4
batch_size = 128
epochs = 10

In [12]:
pretrained = ResNet50(include_top=False, weights='imagenet', input_shape=(150,150,3))
pretrained.summary()

94773248/94765736 [==============================] - 2s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1_conv[0][0]                 
__________________________

In [13]:
# pretrained.output_shape

In [14]:
resnet_features_train = pretrained.predict(x_train)
resnet_features_test = pretrained.predict(x_test)

In [15]:
train_target = to_categorical(y_train,4)

In [16]:
#creating the model
model = Sequential()
# model.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')
model.add(Flatten(input_shape=(5,5,2048)))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
# model.add(BatchNormalization())
model.add(Dense(num_classes, activation = "softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 51200)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               13107456  
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 1028      
Total params: 13,108,484
Trainable params: 13,108,484
Non-trainable params: 0
_________________________________________________________________


In [17]:
#compiling the model
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [18]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='loss', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [19]:
model.fit_generator(datagen.flow(resnet_features_train,train_target, batch_size=batch_size, shuffle=False),
                              epochs = epochs,                             
                              callbacks=[learning_rate_reduction], workers=5)
# model.fit(resnet_features_train, train_target, batch_size=batch_size, epochs=epochs, callbacks=[learning_rate_reduction])

/opt/conda/lib/python3.6/site-packages/keras_preprocessing/image/numpy_array_iterator.py:127: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (5983, 5, 5, 2048) (2048 channels).
  str(self.x.shape[channels_axis]) + ' channels).')


Train for 47 steps
Epoch 1/10
47/47 [==============================] - 770s 16s/step - loss: 4.0608 - accuracy: 0.5646
Epoch 2/10
47/47 [==============================] - 956s 20s/step - loss: 0.8441 - accuracy: 0.6701
Epoch 3/10
47/47 [==============================] - 917s 20s/step - loss: 0.7490 - accuracy: 0.7100
Epoch 4/10
47/47 [==============================] - 933s 20s/step - loss: 0.7025 - accuracy: 0.7269
Epoch 5/10
47/47 [==============================] - 933s 20s/step - loss: 0.6542 - accuracy: 0.7381
Epoch 6/10
47/47 [==============================] - 930s 20s/step - loss: 0.6258 - accuracy: 0.7570
Epoch 7/10
47/47 [==============================] - 929s 20s/step - loss: 0.5780 - accuracy: 0.7737
Epoch 8/10
47/47 [==============================] - 930s 20s/step - loss: 0.5730 - accuracy: 0.7714
Epoch 9/10
47/47 [==============================] - 928s 20s/step - loss: 0.5483 - accuracy: 0.7804
Epoch 10/10
47/47 [==============================] - 926s 20s/step - loss: 0.4975

In [20]:
y_test = model.predict(resnet_features_test)

In [21]:
# y_test = model.predict(x_test)
label = [np.argmax(i) for i in y_test]
y_test = le.inverse_transform(label)
print(y_test)
submission = pd.DataFrame({'Image':test_data.Image, 'Class':y_test})
submission.to_csv('submission.csv',index=False)

['Food' 'Food' 'Food' ... 'misc' 'misc' 'misc']
